In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
%matplotlib inline
import folium
from sklearn.preprocessing import LabelEncoder
from geopy.geocoders import Yandex
from geopy.distance import vincenty

In [34]:
train = pd.read_csv('data/train_set.csv')
test = pd.read_csv('data/test_set.csv')

train.rename(columns={ 'pos_adress_lat' : 'pos_address_lat', 
                       'pos_adress_lon' : 'pos_address_lon'}, inplace=True)

D:\Miniconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (1,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
D:\Miniconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (1,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
# Заполнение пропуска в домашнем адресе одного пользователя

In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1224734 entries, 0 to 1224733
Data columns (total 18 columns):
amount              1224734 non-null float64
atm_address         269473 non-null object
atm_address_lat     184680 non-null float64
atm_address_lon     184680 non-null float64
city                1224725 non-null object
country             1224734 non-null object
currency            1224690 non-null float64
customer_id         1224734 non-null object
home_add_lat        1224718 non-null float64
home_add_lon        1224718 non-null float64
mcc                 1224734 non-null int64
pos_address         924991 non-null object
pos_address_lat     942614 non-null float64
pos_address_lon     942614 non-null float64
terminal_id         1224705 non-null object
transaction_date    1224690 non-null object
work_add_lat        664660 non-null float64
work_add_lon        664660 non-null float64
dtypes: float64(10), int64(1), object(7)
memory usage: 168.2+ MB


In [4]:
train = train.fillna(value=0)

In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1224734 entries, 0 to 1224733
Data columns (total 18 columns):
amount              1224734 non-null float64
atm_address         1224734 non-null object
atm_address_lat     1224734 non-null float64
atm_address_lon     1224734 non-null float64
city                1224734 non-null object
country             1224734 non-null object
currency            1224734 non-null float64
customer_id         1224734 non-null object
home_add_lat        1224734 non-null float64
home_add_lon        1224734 non-null float64
mcc                 1224734 non-null int64
pos_address         1224734 non-null object
pos_address_lat     1224734 non-null float64
pos_address_lon     1224734 non-null float64
terminal_id         1224734 non-null object
transaction_date    1224734 non-null object
work_add_lat        1224734 non-null float64
work_add_lon        1224734 non-null float64
dtypes: float64(10), int64(1), object(7)
memory usage: 168.2+ MB


In [10]:
noaddr = train[(train['atm_address']==0) & (train['atm_address_lat']==0) & (train['pos_address']==0) & (train['pos_address_lat']==0)]
noaddr.shape

(10165, 18)

In [11]:
noaddr[:5]

,amount,atm_address,atm_address_lat,atm_address_lon,city,country,currency,customer_id,home_add_lat,home_add_lon,mcc,pos_address,pos_address_lat,pos_address_lon,terminal_id,transaction_date,work_add_lat,work_add_lon
302067,3.836768,0,0.0,0.0,"""Caf&#233","-bar Campus""",0.0,6e39b7f484135d53815b104d51fd824f,55.901,37.401,7000,0,0.0,0.0,d41d8cd98f00b204e9800998ecf8427e,0,55.533,37.618
555784,2.366405,0,0.0,0.0,"""OSN.PR","ZZ""",0.0,2913d303a872878e726506bf23232fe7,55.639,37.611,235,0,0.0,0.0,d41d8cd98f00b204e9800998ecf8427e,0,0.000,0.000
555788,2.243972,0,0.0,0.0,"""OSN.PR","ZZ""",0.0,2913d303a872878e726506bf23232fe7,55.639,37.611,169,0,0.0,0.0,d41d8cd98f00b204e9800998ecf8427e,0,0.000,0.000
555795,2.487648,0,0.0,0.0,"""OSN.PR","ZZ""",0.0,2913d303a872878e726506bf23232fe7,55.639,37.611,304,0,0.0,0.0,d41d8cd98f00b204e9800998ecf8427e,0,0.000,0.000
555797,2.260354,0,0.0,0.0,"""OSN.PR","ZZ""",0.0,2913d303a872878e726506bf23232fe7,55.639,37.611,183,0,0.0,0.0,d41d8cd98f00b204e9800998ecf8427e,0,0.000,0.000


### Клиенты без адресов

In [16]:
nohome = train[(train['home_add_lat']==0) | (train['home_add_lon']==0)]
print(nohome.shape)
nohome

(16, 18)


,amount,atm_address,atm_address_lat,atm_address_lon,city,country,currency,customer_id,home_add_lat,home_add_lon,mcc,pos_address,pos_address_lat,pos_address_lon,terminal_id,transaction_date,work_add_lat,work_add_lon
766934,2.633742,0,0.0,0.0,NOVOROSSIYSK,RUS,643.0,1a7ea74837f017610970ae508dcd0c26,0.0,0.0,5912,224 DZERZHINSKOGO STRNOVOROSSIYSK353900 RUSRUS,44.675740,37.780223,bc813e84236449ba2da8b055f0569cca,2017-10-15,44.734,37.799
766936,3.255411,0,0.0,0.0,NOVOROSSIYSK,RUS,643.0,1a7ea74837f017610970ae508dcd0c26,0.0,0.0,5411,1 GRIGOREVA STRNOVOROSSIYSK353900 RUSRUS,44.669672,37.782206,d556b96a0e9200444176ca6e26daf0df,2017-08-03,44.734,37.799
766938,3.128106,0,0.0,0.0,NOVOROSSIYSK,RUS,643.0,1a7ea74837f017610970ae508dcd0c26,0.0,0.0,5411,1 GRIGOREVA STRNOVOROSSIYSK353900 RUSRUS,44.669672,37.782206,9fe8e8526fc97f4bacb15d005781244a,2017-04-16,44.734,37.799
766940,3.751279,0,0.0,0.0,Novorossiysk,RUS,643.0,1a7ea74837f017610970ae508dcd0c26,0.0,0.0,5211,"ul.Pionerskay,d.23a,k.2Novorossiysk353922 R...",54.949031,20.238433,e1019fa66b48fffa8b86a7dd52df56da,2017-04-17,44.734,37.799
766942,2.836951,0,0.0,0.0,ENEM,RUS,643.0,1a7ea74837f017610970ae508dcd0c26,0.0,0.0,5983,22 KRASNODAR-NOVOROSSIYSK SHENEM385130 RUSRUS,44.949810,38.927252,2171958b05dfd3459422caef8d03f7d7,2017-07-07,44.734,37.799
766944,3.273269,0,0.0,0.0,NOVOROSSIYSK,RUS,643.0,1a7ea74837f017610970ae508dcd0c26,0.0,0.0,5411,48 LENINA PR-TNOVOROSSIYSK350000 RUSRUS,44.691961,37.789117,2497784a122d19726282990aeb28e903,2017-05-19,44.734,37.799
766946,2.018740,0,0.0,0.0,NOVOROSSIYSK,RUS,643.0,1a7ea74837f017610970ae508dcd0c26,0.0,0.0,5411,204-21 DZERZHINSKOGO PR-TNOVOROSSIYSK353900 ...,44.677703,37.779727,20e23810eb6c445399b5305f861ffe42,2017-08-26,44.734,37.799
766948,2.742684,0,0.0,0.0,NOVOROSSIIYSK,RUS,643.0,1a7ea74837f017610970ae508dcd0c26,0.0,0.0,5411,"217, DZERZHINSKOGONOVOROSSIIYSK353915 RUSRUS",44.678495,37.779635,5f27dff14a6e767aab6958c16ffcff7b,2017-04-17,44.734,37.799
766950,1.785204,0,0.0,0.0,NOVOROSSIYSK,RUS,643.0,1a7ea74837f017610970ae508dcd0c26,0.0,0.0,5912,210 DZERZHINSKOGO PR-TNOVOROSSIYSK353900 RU...,44.678676,37.780864,aaec6ed5a1ff28d06b677ccbad2bf947,2017-04-17,44.734,37.799
766952,2.878566,0,0.0,0.0,NOVOROSSIYSK,RUS,643.0,1a7ea74837f017610970ae508dcd0c26,0.0,0.0,5411,48 LENINA PR-TNOVOROSSIYSK350000 RUSRUS,44.691961,37.789117,ce9f11ab06a343e8082568428bef807c,2017-08-17,44.734,37.799


In [17]:
train[train['customer_id'] == '1a7ea74837f017610970ae508dcd0c26']

,amount,atm_address,atm_address_lat,atm_address_lon,city,country,currency,customer_id,home_add_lat,home_add_lon,mcc,pos_address,pos_address_lat,pos_address_lon,terminal_id,transaction_date,work_add_lat,work_add_lon
766934,2.633742,0,0.000,0.000,NOVOROSSIYSK,RUS,643.0,1a7ea74837f017610970ae508dcd0c26,0.000,0.000,5912,224 DZERZHINSKOGO STRNOVOROSSIYSK353900 RUSRUS,44.675740,37.780223,bc813e84236449ba2da8b055f0569cca,2017-10-15,44.734,37.799
766935,2.633742,0,0.000,0.000,NOVOROSSIYSK,RUS,643.0,1a7ea74837f017610970ae508dcd0c26,44.675,37.781,5912,224 DZERZHINSKOGO STRNOVOROSSIYSK353900 RUSRUS,44.675740,37.780223,bc813e84236449ba2da8b055f0569cca,2017-10-15,44.734,37.799
766936,3.255411,0,0.000,0.000,NOVOROSSIYSK,RUS,643.0,1a7ea74837f017610970ae508dcd0c26,0.000,0.000,5411,1 GRIGOREVA STRNOVOROSSIYSK353900 RUSRUS,44.669672,37.782206,d556b96a0e9200444176ca6e26daf0df,2017-08-03,44.734,37.799
766937,3.255411,0,0.000,0.000,NOVOROSSIYSK,RUS,643.0,1a7ea74837f017610970ae508dcd0c26,44.675,37.781,5411,1 GRIGOREVA STRNOVOROSSIYSK353900 RUSRUS,44.669672,37.782206,d556b96a0e9200444176ca6e26daf0df,2017-08-03,44.734,37.799
766938,3.128106,0,0.000,0.000,NOVOROSSIYSK,RUS,643.0,1a7ea74837f017610970ae508dcd0c26,0.000,0.000,5411,1 GRIGOREVA STRNOVOROSSIYSK353900 RUSRUS,44.669672,37.782206,9fe8e8526fc97f4bacb15d005781244a,2017-04-16,44.734,37.799
766939,3.128106,0,0.000,0.000,NOVOROSSIYSK,RUS,643.0,1a7ea74837f017610970ae508dcd0c26,44.675,37.781,5411,1 GRIGOREVA STRNOVOROSSIYSK353900 RUSRUS,44.669672,37.782206,9fe8e8526fc97f4bacb15d005781244a,2017-04-16,44.734,37.799
766940,3.751279,0,0.000,0.000,Novorossiysk,RUS,643.0,1a7ea74837f017610970ae508dcd0c26,0.000,0.000,5211,"ul.Pionerskay,d.23a,k.2Novorossiysk353922 R...",54.949031,20.238433,e1019fa66b48fffa8b86a7dd52df56da,2017-04-17,44.734,37.799
766941,3.751279,0,0.000,0.000,Novorossiysk,RUS,643.0,1a7ea74837f017610970ae508dcd0c26,44.675,37.781,5211,"ul.Pionerskay,d.23a,k.2Novorossiysk353922 R...",54.949031,20.238433,e1019fa66b48fffa8b86a7dd52df56da,2017-04-17,44.734,37.799
766942,2.836951,0,0.000,0.000,ENEM,RUS,643.0,1a7ea74837f017610970ae508dcd0c26,0.000,0.000,5983,22 KRASNODAR-NOVOROSSIYSK SHENEM385130 RUSRUS,44.949810,38.927252,2171958b05dfd3459422caef8d03f7d7,2017-07-07,44.734,37.799
766943,2.836951,0,0.000,0.000,ENEM,RUS,643.0,1a7ea74837f017610970ae508dcd0c26,44.675,37.781,5983,22 KRASNODAR-NOVOROSSIYSK SHENEM385130 RUSRUS,44.949810,38.927252,2171958b05dfd3459422caef8d03f7d7,2017-07-07,44.734,37.799


In [50]:
df = pd.DataFrame([[np.nan, 2, np.nan, 0],
                    [3, 4, np.nan, 1],
                    [np.nan, np.nan, np.nan, 5],
                    [np.nan, 3, np.nan, 4]],
                    columns=list('ABCD'))

In [51]:
df

,A,B,C,D
0,NaN,2.0,NaN,0
1,3.0,4.0,NaN,1
2,NaN,NaN,NaN,5
3,NaN,3.0,NaN,4


In [45]:
df.fillna(9)

,A,B,C,D
0,9.0,2.0,9.0,0
1,3.0,4.0,9.0,1
2,9.0,9.0,9.0,5
3,9.0,3.0,9.0,4


In [48]:
df.fillna(9, inplace=True)

In [49]:
df

,A,B,C,D
0,9.0,2.0,9.0,0
1,3.0,4.0,9.0,1
2,9.0,9.0,9.0,5
3,9.0,3.0,9.0,4


In [52]:
df['A'].fillna(9, inplace=True)

In [53]:
df

,A,B,C,D
0,9.0,2.0,NaN,0
1,3.0,4.0,NaN,1
2,9.0,NaN,NaN,5
3,9.0,3.0,NaN,4


In [55]:
train['home_add_lat'][train['customer_id'] == '1a7ea74837f017610970ae508dcd0c26'].fillna(value=100, inplace=True)

In [57]:
train[train['customer_id'] == '1a7ea74837f017610970ae508dcd0c26']

,amount,atm_address,atm_address_lat,atm_address_lon,city,country,currency,customer_id,home_add_lat,home_add_lon,mcc,pos_address,pos_address_lat,pos_address_lon,terminal_id,transaction_date,work_add_lat,work_add_lon
766934,2.633742,NaN,NaN,NaN,NOVOROSSIYSK,RUS,643.0,1a7ea74837f017610970ae508dcd0c26,NaN,NaN,5912,224 DZERZHINSKOGO STRNOVOROSSIYSK353900 RUSRUS,44.675740,37.780223,bc813e84236449ba2da8b055f0569cca,2017-10-15,44.734,37.799
766935,2.633742,NaN,NaN,NaN,NOVOROSSIYSK,RUS,643.0,1a7ea74837f017610970ae508dcd0c26,44.675,37.781,5912,224 DZERZHINSKOGO STRNOVOROSSIYSK353900 RUSRUS,44.675740,37.780223,bc813e84236449ba2da8b055f0569cca,2017-10-15,44.734,37.799
766936,3.255411,NaN,NaN,NaN,NOVOROSSIYSK,RUS,643.0,1a7ea74837f017610970ae508dcd0c26,NaN,NaN,5411,1 GRIGOREVA STRNOVOROSSIYSK353900 RUSRUS,44.669672,37.782206,d556b96a0e9200444176ca6e26daf0df,2017-08-03,44.734,37.799
766937,3.255411,NaN,NaN,NaN,NOVOROSSIYSK,RUS,643.0,1a7ea74837f017610970ae508dcd0c26,44.675,37.781,5411,1 GRIGOREVA STRNOVOROSSIYSK353900 RUSRUS,44.669672,37.782206,d556b96a0e9200444176ca6e26daf0df,2017-08-03,44.734,37.799
766938,3.128106,NaN,NaN,NaN,NOVOROSSIYSK,RUS,643.0,1a7ea74837f017610970ae508dcd0c26,NaN,NaN,5411,1 GRIGOREVA STRNOVOROSSIYSK353900 RUSRUS,44.669672,37.782206,9fe8e8526fc97f4bacb15d005781244a,2017-04-16,44.734,37.799
766939,3.128106,NaN,NaN,NaN,NOVOROSSIYSK,RUS,643.0,1a7ea74837f017610970ae508dcd0c26,44.675,37.781,5411,1 GRIGOREVA STRNOVOROSSIYSK353900 RUSRUS,44.669672,37.782206,9fe8e8526fc97f4bacb15d005781244a,2017-04-16,44.734,37.799
766940,3.751279,NaN,NaN,NaN,Novorossiysk,RUS,643.0,1a7ea74837f017610970ae508dcd0c26,NaN,NaN,5211,"ul.Pionerskay,d.23a,k.2Novorossiysk353922 R...",54.949031,20.238433,e1019fa66b48fffa8b86a7dd52df56da,2017-04-17,44.734,37.799
766941,3.751279,NaN,NaN,NaN,Novorossiysk,RUS,643.0,1a7ea74837f017610970ae508dcd0c26,44.675,37.781,5211,"ul.Pionerskay,d.23a,k.2Novorossiysk353922 R...",54.949031,20.238433,e1019fa66b48fffa8b86a7dd52df56da,2017-04-17,44.734,37.799
766942,2.836951,NaN,NaN,NaN,ENEM,RUS,643.0,1a7ea74837f017610970ae508dcd0c26,NaN,NaN,5983,22 KRASNODAR-NOVOROSSIYSK SHENEM385130 RUSRUS,44.949810,38.927252,2171958b05dfd3459422caef8d03f7d7,2017-07-07,44.734,37.799
766943,2.836951,NaN,NaN,NaN,ENEM,RUS,643.0,1a7ea74837f017610970ae508dcd0c26,44.675,37.781,5983,22 KRASNODAR-NOVOROSSIYSK SHENEM385130 RUSRUS,44.949810,38.927252,2171958b05dfd3459422caef8d03f7d7,2017-07-07,44.734,37.799


In [56]:
train['home_add_lat'][train['customer_id'] == '1a7ea74837f017610970ae508dcd0c26']

766934        NaN
766935     44.675
766936        NaN
766937     44.675
766938        NaN
766939     44.675
766940        NaN
766941     44.675
766942        NaN
766943     44.675
766944        NaN
766945     44.675
766946        NaN
766947     44.675
766948        NaN
766949     44.675
766950        NaN
766951     44.675
766952        NaN
766953        NaN
766954     44.675
766955        NaN
766956     44.675
766957     44.675
766958        NaN
766959     44.675
766960        NaN
766961     44.675
766962        NaN
766963     44.675
766964        NaN
766965     44.675
979790     44.675
979791     44.675
979792     44.675
979793     44.675
979794     44.675
979795     44.675
979796     44.675
979797     44.675
979798     44.675
979799     44.675
1012094    44.675
1012095    44.675
1012096    44.675
1012097    44.675
1038302    44.675
1097559    44.675
1097566    44.675
1097586    44.675
1137422    44.675
1158547    44.675
1205930    44.675
1214957    44.675
1214959    44.675
1214960   

In [26]:
# ненулевое значение home_add_lat
train[(train['customer_id'] == '1a7ea74837f017610970ae508dcd0c26') & (train['home_add_lat'] != 0 )]['home_add_lat'].values[0]

44.674999999999997

In [27]:
# ненулевое значение home_add_lon
train[(train['customer_id'] == '1a7ea74837f017610970ae508dcd0c26') & (train['home_add_lon'] != 0 )]['home_add_lon'].values[0]

37.780999999999999

In [31]:
nowork = train[(train['work_add_lat']==0) | (train['work_add_lon']==0)]
print(nowork.shape)
nowork[:5]

(560074, 18)


,amount,atm_address,atm_address_lat,atm_address_lon,city,country,currency,customer_id,home_add_lat,home_add_lon,mcc,pos_address,pos_address_lat,pos_address_lon,terminal_id,transaction_date,work_add_lat,work_add_lon
11,0.299632,0,0.0,0.0,NOVOSIBIRSK,RUS,643.0,0e4c72f1c28452fe3bab0283381b8afd,54.993,82.846,5814,5 KARLA MARKSA SQNOVOSIBIRSK630000 RUSRUS,54.982358,82.892559,c3191e93bafcd29af9c2e4e9e00e9e9e,2017-06-16,0.0,0.0
12,3.173338,0,0.0,0.0,NOVOSIBIRSK,RUS,643.0,0e4c72f1c28452fe3bab0283381b8afd,54.993,82.846,5812,17 KRASNYY PR-TNOVOSIBIRSK630000 RUSRUS,55.026997,82.920634,1d1e7eeadcd06160f3b10d91ff49eba5,2017-07-16,0.0,0.0
13,2.636277,0,0.0,0.0,NOVOSIBIRSK,RUS,643.0,0e4c72f1c28452fe3bab0283381b8afd,54.993,82.846,5814,11-3 VATUTINA STRNOVOSIBIRSK630000 RUSRUS,54.964984,82.928976,8d3284bfe53d66019f4a48863bc24143,2017-05-13,0.0,0.0
14,2.774078,0,0.0,0.0,NOVOSIBIRSK,RUS,643.0,0e4c72f1c28452fe3bab0283381b8afd,54.993,82.846,5813,"UL.KAMENSKAYA,1ANOVOSIBIRSK630099 RUSRUS",55.025451,82.926714,e8e57b788024d5cc1320b7d34a8fe6c1,2017-09-27,0.0,0.0
15,2.764726,0,0.0,0.0,NOVOSIBIRSK,RUS,643.0,0e4c72f1c28452fe3bab0283381b8afd,54.993,82.846,5499,40 KOTOVSKOGO STRNOVOSIBIRSK630108 RUSRUS,54.988900,82.862242,9253b6b81dbfc7e50ed34584a191a5b3,2017-05-28,0.0,0.0


In [32]:
train[train['customer_id'] == '0e4c72f1c28452fe3bab0283381b8afd']

,amount,atm_address,atm_address_lat,atm_address_lon,city,country,currency,customer_id,home_add_lat,home_add_lon,mcc,pos_address,pos_address_lat,pos_address_lon,terminal_id,transaction_date,work_add_lat,work_add_lon
11,0.299632,0,0.000,0.000,NOVOSIBIRSK,RUS,643.0,0e4c72f1c28452fe3bab0283381b8afd,54.993,82.846,5814,5 KARLA MARKSA SQNOVOSIBIRSK630000 RUSRUS,54.982358,82.892559,c3191e93bafcd29af9c2e4e9e00e9e9e,2017-06-16,0.0,0.0
12,3.173338,0,0.000,0.000,NOVOSIBIRSK,RUS,643.0,0e4c72f1c28452fe3bab0283381b8afd,54.993,82.846,5812,17 KRASNYY PR-TNOVOSIBIRSK630000 RUSRUS,55.026997,82.920634,1d1e7eeadcd06160f3b10d91ff49eba5,2017-07-16,0.0,0.0
13,2.636277,0,0.000,0.000,NOVOSIBIRSK,RUS,643.0,0e4c72f1c28452fe3bab0283381b8afd,54.993,82.846,5814,11-3 VATUTINA STRNOVOSIBIRSK630000 RUSRUS,54.964984,82.928976,8d3284bfe53d66019f4a48863bc24143,2017-05-13,0.0,0.0
14,2.774078,0,0.000,0.000,NOVOSIBIRSK,RUS,643.0,0e4c72f1c28452fe3bab0283381b8afd,54.993,82.846,5813,"UL.KAMENSKAYA,1ANOVOSIBIRSK630099 RUSRUS",55.025451,82.926714,e8e57b788024d5cc1320b7d34a8fe6c1,2017-09-27,0.0,0.0
15,2.764726,0,0.000,0.000,NOVOSIBIRSK,RUS,643.0,0e4c72f1c28452fe3bab0283381b8afd,54.993,82.846,5499,40 KOTOVSKOGO STRNOVOSIBIRSK630108 RUSRUS,54.988900,82.862242,9253b6b81dbfc7e50ed34584a191a5b3,2017-05-28,0.0,0.0
16,2.905196,0,0.000,0.000,NOVOSIBIRSK,RUS,643.0,0e4c72f1c28452fe3bab0283381b8afd,54.993,82.846,5814,29-1 KRASNYY PR-T BLDNOVOSIBIRSK630099 RUSRUS,55.031843,82.919409,6739658c8c081a4fcec3678c547505d5,2017-08-09,0.0,0.0
17,2.082621,0,0.000,0.000,NOVOSIBIRSK,RUS,643.0,0e4c72f1c28452fe3bab0283381b8afd,54.993,82.846,5912,29 KRASNYY PR-TNOVOSIBIRSK630000 RUSRUS,55.031051,82.919158,081f4c2f3fc152fe641e486294c4a458,2017-05-14,0.0,0.0
18,1.289195,0,0.000,0.000,NOVOSIBIRSK,RUS,643.0,0e4c72f1c28452fe3bab0283381b8afd,54.993,82.846,4111,"ST.M.PLOCSHAD LENINA, V2NOVOSIBIRSK630099 R...",53.978282,79.243318,7475f345b829bfc72de5f36b9a0cba31,2017-08-09,0.0,0.0
19,1.318607,0,0.000,0.000,NOVOSIBIRSK,RUS,643.0,0e4c72f1c28452fe3bab0283381b8afd,54.993,82.846,4111,"ST.M.PLOCSHAD LENINA, V2NOVOSIBIRSK630099 R...",53.978282,79.243318,7475f345b829bfc72de5f36b9a0cba31,2017-07-16,0.0,0.0
20,2.849170,0,0.000,0.000,NOVOSIBIRSK,RUS,643.0,0e4c72f1c28452fe3bab0283381b8afd,54.993,82.846,5411,"2,TROLLEYNAYANOVOSIBIRSK630108 RUSRUS",54.970228,82.861248,963873075eda3b9ce1ff4b75717bed47,2017-07-03,0.0,0.0


In [ ]:
# Найти клиентов, у которых нет работы